In [2]:
from pyngrok import ngrok
ngrok.kill()

In [3]:
! ngrok authtoken 1QRvEl1nzfpzo3EFRPbgj1az8CP_3hafyvnxMbEvALxqtLwX7

Authtoken saved to configuration file: C:\Users\smura\.ngrok2\ngrok.yml


In [4]:
ngrok.connect(5000)

t=2023-06-04T10:23:05+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\smura\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\smura\\.ngrok2\\ngrok.yml
t=2023-06-04T10:23:05+0530 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


<NgrokTunnel: "https://52b2-2401-4900-1cd4-45f7-4d59-8efa-a52a-dc03.ngrok-free.app" -> "http://localhost:5000">

In [5]:
!apt-get install portaudio19-dev python3-pyaudio

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
from flask import Flask
import datetime
from flask import request # used to parse payload
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from flask import render_template
from flask import abort
from flask_cors import CORS
import os

# define a variable to hold you app
app = Flask(__name__)
CORS(app)

@app.route('/')
def hello():
    return render_template('index.html')

@app.route('/time', methods=['GET'])
def get_time():
    return str(datetime.datetime.now())

@app.route('/api/summarize', methods=['GET'])
def GetUrl():
    """
    Called as /api/summarize?youtube_url='url'
    """
    # if user sends payload to variable name, get it. Else empty string
    video_url = request.args.get('youtube_url', '') 
    # if(len(video_url) == 0) or (not '=' in video_url):
    #   print("f")
    #   abort(404)
    
    response = GetTranscript(video_url)
    return response

def SumySummarize(text):

    from sumy.parsers.html import HtmlParser
    from sumy.parsers.plaintext import PlaintextParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
    from sumy.nlp.stemmers import Stemmer
    from sumy.utils import get_stop_words

    LANGUAGE = "english"
    SENTENCES_COUNT = 10
    import nltk;  
    nltk.download('punkt')

    # url = "https://en.wikipedia.org/wiki/Automatic_summarization"
    # parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    s = ""
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
      s += (str)(sentence)
      s += "\n"
    print('The summary is: ', s)
    return s

def GetTextFromAudio():
    import speech_recognition as sr
    from pydub import AudioSegment
    
    f = ""

    # convert mp3 file to wav           
    for file in os.listdir(os.getcwd()):
      if file.endswith(".mp3"):
          f = os.path.join(os.getcwd(), file)
          print(os.path.join(os.getcwd(), file))      
                                          
    if(len(f) == 0):
      return f
    print('The file is: ', f)
    sound = AudioSegment.from_mp3(f)

    os.remove(f)
    
    sound.export("transcript.wav", format="wav")
    
    # use the audio file as the audio source                                        
    AUDIO_FILE = "transcript.wav"

    r = sr.Recognizer()
    with sr.AudioFile(AUDIO_FILE) as source:
      audio = r.record(source)  # read the entire audio file    
      print(audio)              
      return (r.recognize_google(audio))

def GetAudio(video_url):
    from youtube_dl import YoutubeDL
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3'
        }],
    }
    with YoutubeDL(ydl_opts) as ydl:
       ydl.download([video_url])
      

# video id are the last characters in the link of youtube video
def GetTranscript(video_url): 
    text = ""
    try:
      video_id = video_url.split('=')[1]
      transcript = YouTubeTranscriptApi.get_transcript(video_id)
      formatter = TextFormatter()
      text = formatter.format_transcript(transcript)
      return SumySummarize(text)
    except:
      GetAudio(video_url)
      text = GetTextFromAudio()
      print('The text is: ', text)
      return SumySummarize(text)

# server the app when this file is run
if __name__ == '__main__': 
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jun/2023 10:23:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 10:23:24] "GET /favicon.ico HTTP/1.1" 404 -
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
127.0.0.1 - - [04/Jun/2023 10:38:02] "GET /api/summarize?youtube_url=https://www.youtube.com/watch?v=eIrMbAQSU34 HTTP/1.1" 200 -


The summary is:  Hi!
My name is Mosh and I'm gonna be your instructor in this Java course.
In this course, you're gonna learn everything you need to get started programming in Java.
We'll start off by installing all the necessary tools to build Java applications then you're gonna learn about the basics of Java you'll learn how Java code gets executed you'll learn how to build simple algorithms and throughout this course I'm gonna share with you lots of tips and shortcuts from my years of experience I'll teach you how to write good code like a professional developer so we'll end up watching this course you will have a solid foundation in Java and be ready to learn about advanced Java features I've designed this course for anyone who wants to learn Java if you're a beginner don't worry I'll make Java super simple and hold your hands through this entire course you're not too old or too young you'll write your first Java program in minutes my name is mosh I'm a software engineer with two d

[2023-06-04 10:38:18,728] ERROR in app: Exception on /api/summarize [GET]
Traceback (most recent call last):
  File "C:\Users\smura\AppData\Local\Temp\ipykernel_9952\2336647980.py", line 116, in GetTranscript
    video_id = video_url.split('=')[1]
               ~~~~~~~~~~~~~~~~~~~~^^^
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\smura\AppData\Roaming\Python\Python311\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                            

The summary is:  Hi!
My name is Mosh and I'm gonna be your instructor in this Java course.
In this course, you're gonna learn everything you need to get started programming in Java.
We'll start off by installing all the necessary tools to build Java applications then you're gonna learn about the basics of Java you'll learn how Java code gets executed you'll learn how to build simple algorithms and throughout this course I'm gonna share with you lots of tips and shortcuts from my years of experience I'll teach you how to write good code like a professional developer so we'll end up watching this course you will have a solid foundation in Java and be ready to learn about advanced Java features I've designed this course for anyone who wants to learn Java if you're a beginner don't worry I'll make Java super simple and hold your hands through this entire course you're not too old or too young you'll write your first Java program in minutes my name is mosh I'm a software engineer with two d

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
127.0.0.1 - - [04/Jun/2023 10:51:39] "GET /api/summarize?youtube_url="https://www.youtube.com/watch?v=eIrMbAQSU34" HTTP/1.1" 200 -


The summary is:  Hi!
My name is Mosh and I'm gonna be your instructor in this Java course.
In this course, you're gonna learn everything you need to get started programming in Java.
We'll start off by installing all the necessary tools to build Java applications then you're gonna learn about the basics of Java you'll learn how Java code gets executed you'll learn how to build simple algorithms and throughout this course I'm gonna share with you lots of tips and shortcuts from my years of experience I'll teach you how to write good code like a professional developer so we'll end up watching this course you will have a solid foundation in Java and be ready to learn about advanced Java features I've designed this course for anyone who wants to learn Java if you're a beginner don't worry I'll make Java super simple and hold your hands through this entire course you're not too old or too young you'll write your first Java program in minutes my name is mosh I'm a software engineer with two d

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
127.0.0.1 - - [04/Jun/2023 10:52:39] "GET /api/summarize?youtube_url="https://www.youtube.com/watch?v=9s29LKfEFjQ" HTTP/1.1" 200 -


The summary is:  this is how i would learn to code if i could start over for context i work at microsoft but before that i did six internships across three countries so yeah i've done a line or two in my life wait not like that what the to tell you what i would change i first need to tell you what i did i started coding in grade 11. well sorta we started with this language called scratch it's basic game development made for children nobody would be stupid enough to let's say do an entire google coding interview in this language and maybe even do shots every time the code fails so let's just skip that and get to the important stuff i went to waterloo and in order i learned first racket then c c plus plus html css tcl python java javascript c sharp some of these were in class some are on their own here's the motivation behind each racket for recursion c for memory management c plus plus for object oriented design html and css for web development although these are not real programming la